In [118]:
from selenium.webdriver.common.action_chains import ActionChains
#from optparse import OptionParser
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import sys
import re

pattern_name = "\\n(.+)\\n"  # Used to extract names
pattern_headline = 'occupation\\n(.+)\\n'  # Used to extract headlines


In [119]:
def login(email, password):
    driver = webdriver.Chrome('./chromedriver')
    driver.get("https://www.linkedin.com")
    print(email)
    print(password)
    session_key = driver.find_element_by_name("session_key")
    session_key.send_keys(email)
    session_password = driver.find_element_by_name("session_password")
    session_password.send_keys(password)
    submit = driver.find_element_by_class_name("sign-in-form__submit-button")
    submit.click()
    #print(driver.title)
    if "LinkedIn" not in driver.title:
        print("Provided E-mail/Password is wrong!")
        driver.quit()
        sys.exit()
  
    return driver

In [120]:
def scrap_basic(driver):
    driver.get("https://www.linkedin.com/mynetwork/invite-connect/connections/")
    time_to_wait = 3  
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        for i in range(2):
            time.sleep(time_to_wait)
            driver.execute_script("window.scrollTo(0, 0);")  
            time.sleep(time_to_wait)
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        new_height = driver.execute_script("return document.body.scrollHeight")  
        if new_height == last_height:
            break
        last_height = new_height
    extracted_scrap = driver.find_elements_by_class_name("mn-connection-card__details")
    extracted_scrap = [_.text for _ in extracted_scrap]
   
    names = []
    headlines = []
    for card in extracted_scrap:
        try:
            names.append(re.search(pattern_name, card)[0])
            #print(re.search(pattern_name, card)[0])
        except:
            names.append(" ")
        try:
            headlines.append(re.search(pattern_headline, card)[0])
            #print(re.search(pattern_headline, card)[0])
        except:
            headlines.append(" ")
    extracted_scrap = driver.find_elements_by_tag_name('a')
    links = []
    for i in extracted_scrap:
        link = i.get_attribute("href")
        if "https://www.linkedin.com/in" in link and not link in links:
            links.append(link)
    
    return driver, names, headlines, links

In [121]:
def scrap_companies(driver, links):
    companies = []
    length = len(links)
    print(len(links))
    for i in range(length):
        link = links[i]  
        driver.get(link)
        time_to_wait = 3
        last_height = driver.execute_script("return document.body.scrollHeight")
        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            for i in range(2):
                time.sleep(time_to_wait)
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight/4);")
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight/3);")
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight/2);")
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight*3/4);")
                time.sleep(time_to_wait)
                
                driver.execute_script(
                    "window.scrollTo(0, document.body.scrollHeight);")

            new_height = driver.execute_script("return document.body.scrollHeight")  
            if new_height == last_height:
                break
            last_height = new_height

        print(link)
#         buttons = driver.find_elements_by_tag_name("button")
#         lengthb = len(buttons)
#         print(lengthb)
        
        #works = driver.find_element_by_class_name("inline-show-more-text inline-show-more-text--is-collapsed inline-show-more-text--is-collapsed-with-line-clamp inline")
        try:
            works = driver.find_element_by_xpath("//div[starts-with(@aria-label, 'Current company')]").get_attribute("innerHTML")
            companies.append(works)
            #print(re.search(pattern_name, card)[0])
        except:
            companies.append("Not working")
    return driver,companies   

In [122]:
def save_to_csv(names, headlines, links, comps):
    if comps is None:
        comps = [None]*len(names)
    df = pd.DataFrame()
    for i in range(len(names)-1):
        #print(names[i])
        df = df.append({"Name": names[i], "Headline": headlines[i],"Link": links[i], "Companies": skills[i]}, ignore_index=True)
    #print(df.head())
    df.to_csv("final_dataComp.csv", index=False, columns=["Name", "Headline", "Link", "Companies"])

In [123]:
if __name__ == "__main__":
    #(options, args) = parser.parse_args()

    # Inputs
    email = 'osjyothi156@gmail.com'
    password = 'Wksfgagfagfag5S6' # Not original
    companies = 's'

    driver = login(email, password) 
    print("Successfull Login!")
    print("Scrapping connections...")
    driver, names, headlines, links = scrap_basic(driver)  
    print("Finished scraping basic data, scrapped {}".format(len(names)))
    
    

    if companies:
        print("Company names scraping start...")
        driver, comps = scrap_companies(driver, links)  
        print("Finished Companies scrap.")
        print("Saving to CSV file...")
        compdf = save_to_csv(names, headlines, links, comps)  
    else:
        save_to_csv(names, headlines, links, None)  

    print("Scrapping session has ended.")
    driver.quit()

<ipython-input-119-0ec8bf917421>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver')


osjyothi156@gmail.com
WkV6edKmpUGj5S6


<ipython-input-119-0ec8bf917421>:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  session_key = driver.find_element_by_name("session_key")
<ipython-input-119-0ec8bf917421>:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  session_password = driver.find_element_by_name("session_password")
<ipython-input-119-0ec8bf917421>:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  submit = driver.find_element_by_class_name("sign-in-form__submit-button")


Successfull Login!
Scrapping connections...


<ipython-input-120-365daf562ed3>:17: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  extracted_scrap = driver.find_elements_by_class_name("mn-connection-card__details")
<ipython-input-120-365daf562ed3>:33: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  extracted_scrap = driver.find_elements_by_tag_name('a')


Finished basic scrap, scrapped 47
Company names scraping start...
47
https://www.linkedin.com/in/mallika-ambati-06252978/


<ipython-input-121-f240373fbec8>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  works = driver.find_element_by_xpath("//div[starts-with(@aria-label, 'Current company')]").get_attribute("innerHTML")


https://www.linkedin.com/in/akiraspann/
https://www.linkedin.com/in/jdye/
https://www.linkedin.com/in/tim-mckinney-4264b6166/
https://www.linkedin.com/in/shy-mccarver-90a838112/
https://www.linkedin.com/in/arshawnnawhite/
https://www.linkedin.com/in/adekoyejo-odedere-1b7771190/
https://www.linkedin.com/in/sitaram-bonduli-ceng-mimeche-630a3422/
https://www.linkedin.com/in/natalie-forte/
https://www.linkedin.com/in/nikhila-simhadri/
https://www.linkedin.com/in/jake-schaller-547245109/
https://www.linkedin.com/in/gayathri-dondeti-a16926161/
https://www.linkedin.com/in/denzel-prince-4899b71b9/
https://www.linkedin.com/in/smunagala24/
https://www.linkedin.com/in/adityanival/
https://www.linkedin.com/in/bhavani-prasad-kommi-29bb4a232/
https://www.linkedin.com/in/prachi-shah-43829a196/
https://www.linkedin.com/in/taylor-burgess-99a0981a2/
https://www.linkedin.com/in/hannah-haswell-83140718a/
https://www.linkedin.com/in/diljaan-shaik-313537bb/
https://www.linkedin.com/in/corey-notman-mba-hr-ma